In [1]:
import numpy as np

def load_csv_to_ndarray(filepath):
    # Read using numpy lib and return all rows, no header in this case
    return np.genfromtxt(filepath, dtype=float, delimiter=',')

def transform_feature(one_dim_feature_vector):
    transformed_feature_vector = np.zeros((len(one_dim_feature_vector), 2*128))
    
    for i in range(0, len(one_dim_feature_vector)):
        feature = one_dim_feature_vector[i]
        transformed_feature = np.zeros(2*128)
        
        for c in range(0, 128):
            transformed_feature[2*c] = np.cos((c+1) * feature)
            transformed_feature[(2*c)+1] = np.sin((c+1) * feature)
        
        transformed_feature_vector[i] = transformed_feature
        
    return transformed_feature_vector 

In [2]:
from helper import load_csv_to_ndarray


train_df = load_csv_to_ndarray('hw3p1_train.csv')
test_df = load_csv_to_ndarray('hw3p1_test.csv')

print(train_df.shape)
print(test_df.shape)

(32, 2)
(2079,)


In [3]:
print(train_df[:,0])

[-3.0434   -2.8439   -2.6444   -2.4449   -2.2454   -2.0458   -1.8463
 -1.6468   -1.4473   -1.2478   -1.0483   -0.84874  -0.64922  -0.4497
 -0.25019  -0.050671  0.14885   0.34836   0.54788   0.7474    0.94691
  1.1464    1.3459    1.5455    1.745     1.9445    2.144     2.3435
  2.543     2.7426    2.9421    3.1416  ]


In [4]:
transform_feature(train_df[:,0])

array([[-0.99518297, -0.99518297,  0.9807783 , ..., -0.99540477,
         0.99999738,  0.99999738],
       [-0.95601581, -0.95601581,  0.82793247, ..., -0.99418927,
         0.91888641,  0.91888641],
       [-0.87892502, -0.87892502,  0.54501837, ..., -0.95183999,
         0.69036231,  0.69036231],
       ...,
       [-0.92145281, -0.92145281,  0.69815055, ..., -0.9184769 ,
         0.69269505,  0.69269505],
       [-0.98016725, -0.98016725,  0.92145566, ..., -0.97950127,
         0.92015568,  0.92015568],
       [-1.        , -1.        ,  1.        , ..., -0.99999956,
         0.99999956,  0.99999956]])